In [1]:
import pandas as pd
import re
import random
import numpy as np

In [2]:
df = pd.read_csv('bicicletas/AccidentesBicicletas_2018.csv', encoding='latin_1',sep = ';')

In [3]:
def change_cols_17_18(df):
    dict_cols = {
        'FECHA':'dia',
        'DISTRITO':'distrito', 'LUGAR ACCIDENTE': 'calle', 'Nº': 'num',
        'TIPO ACCIDENTE':'tipo_accidente', 'LESIVIDAD': 'lesividad',
        'CPFA Granizo':'Granizo', 'CPFA Hielo': 'Hielo', 'CPFA Lluvia': 'Lluvia',
        'CPFA Niebla': 'Niebla', 'CPFA Seco': 'Despejado', 'CPFA Nieve': 'Nieve',
    }
    to_drop = ['Nº PARTE','CPSV Seca Y Limpia', 'CPSV Mojada', 'CPSV Aceite', 'CPSV Barro', 'CPSV Grava Suelta', 'CPSV Hielo', '* Nº VICTIMAS', 'Tipo Vehiculo', 'TIPO PERSONA', 'SEXO', 'Tramo Edad']
    df.rename(columns=dict_cols, inplace = True)
    df.drop(to_drop, axis=1, inplace=True)
    
    return df

df = change_cols_17_18(df)

In [4]:
    
def create_weather_17_18 (df):
    lst_w = list(df[df.columns[6:12]])
    for e in lst_w:
        df[e] = df[e].apply(lambda x: '' if (x=='NO') else e)
    df['meteo'] = df[lst_w].sum(axis=1)
    df.drop(lst_w, axis=1, inplace=True)
    return df
    
df = create_weather_17_18(df)

In [5]:
df.head()

,dia,RANGO HORARIO,DIA SEMANA,distrito,calle,num,tipo_accidente,lesividad,meteo
0,01/01/2018,DE 13:00 A 13:59,LUNES,ARGANZUELA,CALLE DE ANCORA - PASEO DE LAS DELICIAS ...,0,COLISIÓN DOBLE,HL,Despejado
1,01/01/2018,DE 15:00 A 15:59,LUNES,CENTRO,CALLE DE ALCALA NUM ...,44,ATROPELLO,HL,Despejado
2,02/01/2018,DE 12:00 A 12:59,MARTES,SALAMANCA,CALLE DE SERRANO - CALLE DEL CONDE DE ARANDA ...,0,ATROPELLO,HL,Despejado
3,03/01/2018,DE 13:00 A 13:59,MIERCOLES,SALAMANCA,PLAZA DE LA INDEPENDENCIA NUM ...,5,COLISIÓN DOBLE,HL,Despejado
4,03/01/2018,DE 15:00 A 15:59,MIERCOLES,CIUDAD LINEAL,CALLE DE RAMIREZ DE ARELLANO NUM ...,35,COLISIÓN DOBLE,HL,Despejado


In [6]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df['fecha'] = df['fecha'].dt.round('H')
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

In [7]:
def create_date_17_18(df):
    df['hora'] = df['RANGO HORARIO'].str.extract(r'(\d[\d]?:\d+\s)')
    create_date(df)
    df.drop(['RANGO HORARIO', 'DIA SEMANA'], axis=1, inplace=True)
    return df

df = create_date_17_18(df)

In [8]:
df.head()

,distrito,calle,num,tipo_accidente,lesividad,meteo,fecha
0,ARGANZUELA,CALLE DE ANCORA - PASEO DE LAS DELICIAS ...,0,COLISIÓN DOBLE,HL,Despejado,2018-01-01 13:00:00
1,CENTRO,CALLE DE ALCALA NUM ...,44,ATROPELLO,HL,Despejado,2018-01-01 15:00:00
2,SALAMANCA,CALLE DE SERRANO - CALLE DEL CONDE DE ARANDA ...,0,ATROPELLO,HL,Despejado,2018-01-02 12:00:00
3,SALAMANCA,PLAZA DE LA INDEPENDENCIA NUM ...,5,COLISIÓN DOBLE,HL,Despejado,2018-01-03 13:00:00
4,CIUDAD LINEAL,CALLE DE RAMIREZ DE ARELLANO NUM ...,35,COLISIÓN DOBLE,HL,Despejado,2018-01-03 15:00:00


In [9]:
def create_dir(df):
    df.num = df.num.apply(lambda x: x.strip())
    df.num = df.num.apply(lambda x: '-' if x=='0' else x)
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion = df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

df = create_dir(df)

In [10]:
def change_accident(values):
    dict_accidente = {
        r'caída':'Caida',
        r'colisión':'Colision',
        r'alcance':'Colision',
        r'atropello':'Imprudencia',
        r'choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values.lower()):
            return v
    return 'Otro'

df.tipo_accidente = df.tipo_accidente.apply(lambda x: change_accident(x))

In [11]:
df.head()

,distrito,tipo_accidente,lesividad,meteo,fecha,direccion
0,ARGANZUELA,Colision,HL,Despejado,2018-01-01 13:00:00,CALLE DE ANCORA - PASEO DE LAS DELICIAS
1,CENTRO,Imprudencia,HL,Despejado,2018-01-01 15:00:00,CALLE DE ALCALA NUM ...
2,SALAMANCA,Imprudencia,HL,Despejado,2018-01-02 12:00:00,CALLE DE SERRANO - CALLE DEL CONDE DE ARANDA
3,SALAMANCA,Colision,HL,Despejado,2018-01-03 13:00:00,PLAZA DE LA INDEPENDENCIA NUM ...
4,CIUDAD LINEAL,Colision,HL,Despejado,2018-01-03 15:00:00,CALLE DE RAMIREZ DE ARELLANO NUM ...


In [12]:
df.lesividad.value_counts(dropna=False)

HL                                          569
IL                                           62
HG                                           60
NO ASIGNADA                                   9
Name: lesividad, dtype: int64

In [13]:
def change_injury_17_18(values):
    dict_injury = {
        'IL':'Moderada',
        'HG':'Grave',
        'M':'Fallecido',
        'HL':'Leve', 'NO ASIGNADA':'Leve'}
    return dict_injury[values]

df.lesividad = df.lesividad.apply(lambda x: change_injury_17_18(x.strip()))

In [14]:
df.head()

,distrito,tipo_accidente,lesividad,meteo,fecha,direccion
0,ARGANZUELA,Colision,Leve,Despejado,2018-01-01 13:00:00,CALLE DE ANCORA - PASEO DE LAS DELICIAS
1,CENTRO,Imprudencia,Leve,Despejado,2018-01-01 15:00:00,CALLE DE ALCALA NUM ...
2,SALAMANCA,Imprudencia,Leve,Despejado,2018-01-02 12:00:00,CALLE DE SERRANO - CALLE DEL CONDE DE ARANDA
3,SALAMANCA,Colision,Leve,Despejado,2018-01-03 13:00:00,PLAZA DE LA INDEPENDENCIA NUM ...
4,CIUDAD LINEAL,Colision,Leve,Despejado,2018-01-03 15:00:00,CALLE DE RAMIREZ DE ARELLANO NUM ...


In [2]:
def get_df_norm_header_index(path):
    lst_header = ['num_exp', 'dia', 'hora', 'calle', 'num', 'distrito', 'tipo_accidente', 'meteo', 'vehiculo', 'persona', 'edad', 'sexo', 'lesividad' ]
    df = pd.read_csv(path, encoding='latin_1',sep = ';', names=lst_header)
    df.drop(0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

bici = get_df_norm_header_index('bicicletas/AccidentesBicicletas_2019.csv')


In [3]:
def create_date(df):
    df['fecha'] = df.dia+' '+df.hora
    df['fecha'] = pd.to_datetime(df['fecha'], dayfirst=True)
    df['fecha'] = df['fecha'].dt.round('H')
    df.drop(['dia', 'hora'], axis=1, inplace = True)
    return df

bici = create_date(bici)

In [4]:
def create_dir(df):
    df.num.apply(lambda x: x.strip())
    df.num = df.num.apply(lambda x: '-' if x=='0' else x)
    df.loc[(df.num == '-'), 'num']=''
    df['direccion'] = df.calle+' '+df.num
    df.direccion.apply(lambda x: x.strip())
    df.drop(['calle', 'num'], axis=1, inplace = True)
    return df

bici = create_dir(bici)

In [5]:
def fill_drop(df):
    df.lesividad.fillna('14', inplace=True)
    df.meteo.fillna('Se desconoce', inplace=True)
    df.drop(['sexo', 'vehiculo', 'persona', 'edad', 'num_exp'], axis=1, inplace=True)
    return df

bici = fill_drop(bici)

In [6]:
def change_accident(values):
    dict_accidente = {
        r'caída':'Caida',
        r'colisión':'Colision',
        r'alcance':'Colision',
        r'atropello':'Imprudencia',
        r'choque':'Imprudencia'}
    for k, v in dict_accidente.items():
        if re.match(k, values.lower()):
            return v
    return 'Otro'

bici.tipo_accidente = bici.tipo_accidente.apply(lambda x: change_accident(x))

In [7]:
def change_injury(values):
    dict_injury = {
        '01':'Moderada','02':'Moderada',
        '03':'Grave',
        '04':'Fallecido',
        '05':'Leve','06':'Leve','07':'Leve','14':'Leve'}
    return dict_injury[values]


bici.lesividad = bici.lesividad.apply(lambda x: change_injury(str(x)))

In [8]:
bici = bici[['fecha','meteo', 'tipo_accidente', 'lesividad', 'distrito', 'direccion']]

In [9]:
bici.head()

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019-01-03 14:00:00,Se desconoce,Colision,Leve,VILLA DE VALLECAS,CALL. FELIPE ALVAREZ 10
3,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [10]:
bici.tipo_accidente.value_counts()

Colision       388
Caida          230
Imprudencia    141
Otro            20
Name: tipo_accidente, dtype: int64

In [ ]:
# encoding='cp850'
# encoding='cp1250'

# **
# encoding='latin_1'
# encoding='iso8859_15'

In [18]:
abr = pd.read_csv('tiempo/Anio2019/abr_meteo19.csv', encoding='latin_1',sep = ';')

In [20]:
abr.head(20)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,102,81,28079102_81_98,2019,4,1,1.90,V,...,1.45,V,0.90,V,0.97,V,1.68,V,1.12,V
1,28,79,102,81,28079102_81_98,2019,4,2,0.40,V,...,2.48,V,2.05,V,1.88,V,0.95,V,1.10,V
2,28,79,102,81,28079102_81_98,2019,4,3,1.18,V,...,3.95,V,3.53,V,3.40,V,3.42,V,2.58,V
3,28,79,102,81,28079102_81_98,2019,4,4,2.35,V,...,3.12,V,2.52,V,2.30,V,2.13,V,1.88,V
4,28,79,102,81,28079102_81_98,2019,4,5,3.02,V,...,2.88,V,1.43,V,0.93,V,2.18,V,2.58,V
5,28,79,102,81,28079102_81_98,2019,4,6,2.18,V,...,4.93,V,2.98,V,3.07,V,1.95,V,1.83,V
6,28,79,102,81,28079102_81_98,2019,4,7,2.77,V,...,1.73,V,1.48,V,0.73,V,0.37,V,0.75,V
7,28,79,102,81,28079102_81_98,2019,4,8,1.18,V,...,3.95,V,2.82,V,2.50,V,4.33,V,1.30,V
8,28,79,102,81,28079102_81_98,2019,4,9,1.22,V,...,3.48,V,3.92,V,3.15,V,2.98,V,3.67,V
9,28,79,102,81,28079102_81_98,2019,4,10,1.85,V,...,2.02,V,1.65,V,1.68,V,1.73,V,1.60,V
